In [4]:
%pip install --user mlflow --quiet
%pip install --user optuna
%pip install --user mlflow scikit-learn
%pip install -U imbalanced-learn
%pip install -U xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import optuna
import gc
import mlflow
import imblearn
import xgboost as xgb

#### Leitura dos dados

In [2]:
data_path = './TRNcod.xls'

In [3]:
df_excel = pd.read_csv(data_path, sep='\t', encoding='utf-8')

In [4]:
print('Quantidade de entradas:', len(df_excel.index))

Quantidade de entradas: 389196


In [5]:
df_excel.head()

,INDEX,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
0,0,1,1,1,0,0,0,0,0.135098,1,...,0,0,1,1,0,1,1,1,0,1
1,1,1,0,1,0,0,1,0,0.273504,1,...,0,1,0,1,1,0,0,0,1,0
2,2,1,0,1,0,0,1,0,0.281910,0,...,1,1,0,0,0,0,1,0,1,0
3,3,1,1,1,0,0,0,0,0.225741,0,...,1,1,0,1,1,0,1,0,1,0
4,4,1,1,0,0,0,1,0,0.480403,0,...,1,1,1,0,0,1,0,1,1,0


In [6]:
df_excel.drop(['INDEX'], axis=1, inplace=True)

#### Combinando classes em uma única coluna

In [7]:
df = df_excel

In [8]:
def label_class (row):
   if row['IND_BOM_1_1'] == 1 and row['IND_BOM_1_2'] == 1:
      return 2
   if row['IND_BOM_1_1'] == 1:
      return 0
   if row['IND_BOM_1_2'] == 1:
      return 1
   return 3

In [9]:
df['class'] = df.apply (lambda row: label_class(row), axis=1)

In [10]:
df.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,1,1,0,1,1,1,0,1,1
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,1,0,1,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,1,0,0,0,0,1,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,1,0,1,1,0,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,0,0,1,0,1,1,0,0


In [11]:
print('classe 0:', len(df[df['class'] == 0]))
print('classe 1:', len(df[df['class'] == 1]))
print('classe desconhecida:', len(df[df['class'] > 1]))

classe 0: 255098
classe 1: 134098
classe desconhecida: 0


#### 1 Etapa - Particionamento dos dados

In [12]:
x_class_0 = df[df['class'] == 0]
x_class_1 = df[df['class'] == 1]

In [13]:
x_class_0.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,1,0,1,1,0,0,0,1,0,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,1,0,0,0,0,1,0,1,0,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,1,0,1,1,0,1,0,1,0,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,0,0,1,0,1,1,0,0
5,0,1,1,0,0,0,1,0.219323,0,0.111111,...,1,1,0,1,0,0,1,1,0,0


In [14]:
x_class_1.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2,class
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,1,1,0,1,1,1,0,1,1
10,1,0,1,1,0,0,0,0.654703,0,0.111111,...,1,0,1,0,0,0,0,0,1,1
11,1,1,1,0,0,0,0,0.097444,1,0.111111,...,1,0,1,0,1,0,1,0,1,1
12,1,0,1,0,1,0,0,0.398961,0,0.111111,...,1,1,0,1,0,0,0,0,1,1
17,1,1,1,0,0,0,0,0.142254,0,0.111111,...,0,0,1,0,1,0,1,0,1,1


#### 2 Etapa - Particionamento dos dados

In [15]:
y_class_0 = x_class_0['class'].values
y_class_1 = x_class_1['class'].values

#### Separar dados em treino, validação e treino

Os dados foram divididos aleatoriamente usando o método train_test_split()
- 50% dos dados para treinamento
- Dos 50% restantes, metade(25% do total) pra validação e o restante(25%) para teste

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train_class_0, X_rem_class_0, y_train_class_0, y_rem_class_0 = train_test_split(x_class_0, y_class_0, train_size=0.5)
X_valid_class_0, X_test_class_0, y_valid_class_0, y_test_class_0 = train_test_split(X_rem_class_0, y_rem_class_0, test_size=0.5)

In [18]:
X_train_class_1, X_rem_class_1, y_train_class_1, y_rem_class_1 = train_test_split(x_class_1, y_class_1, train_size=0.5)
X_valid_class_1, X_test_class_1, y_valid_class_1, y_test_class_1 = train_test_split(X_rem_class_1, y_rem_class_1, test_size=0.5)

In [19]:
X_train = pd.concat([X_train_class_0, X_train_class_1])
y_train = np.concatenate((y_train_class_0, y_train_class_1))
X_train = X_train.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [20]:
X_valid = pd.concat([X_valid_class_0, X_valid_class_1])
y_valid = np.concatenate((y_valid_class_0, y_valid_class_1))
X_valid = X_valid.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [21]:
X_test = pd.concat([X_test_class_0, X_test_class_1])
y_test = np.concatenate((y_test_class_0, y_test_class_1))
X_test = X_test.drop(['class', 'IND_BOM_1_1', 'IND_BOM_1_2'], axis=1)

In [22]:
print('Quantidade de entradas para treino:', len(X_train), len(y_train))
print('Quantidade de entradas para validação:', len(X_valid), len(y_valid))
print('Quantidade de entradas para teste:', len(X_test), len(y_test))

Quantidade de entradas para treino: 194598 194598
Quantidade de entradas para validação: 97298 97298
Quantidade de entradas para teste: 97300 97300


#### Oversampling - Replicar a classe minoritária para ficar do tamanho da classe majoritária

In [23]:
print('Quantidade antes do over sampling')
print('Quantidade classe 0 treino:', (y_train == 0).sum())
print('Quantidade classe 1 treino:', (y_train == 1).sum())
print('Quantidade classe 0 valid:', (y_valid == 0).sum())
print('Quantidade classe 1 valid:', (y_valid == 1).sum())
print('Quantidade classe 0 test:', (y_test == 0).sum())
print('Quantidade classe 1 test:', (y_test == 1).sum())

Quantidade antes do over sampling
Quantidade classe 0 treino: 127549
Quantidade classe 1 treino: 67049
Quantidade classe 0 valid: 63774
Quantidade classe 1 valid: 33524
Quantidade classe 0 test: 63775
Quantidade classe 1 test: 33525


In [24]:
from imblearn.over_sampling import SMOTE

In [25]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [26]:
print('Quantidade classe 0 treino:', (y_train == 0).sum())
print('Quantidade classe 1 treino:', (y_train == 1).sum())

Quantidade classe 0 treino: 127549
Quantidade classe 1 treino: 127549


In [27]:
oversample = SMOTE()
X_valid, y_valid = oversample.fit_resample(X_valid, y_valid)

In [28]:
print('Quantidade classe 0 valid:', (y_valid == 0).sum())
print('Quantidade classe 1 valid:', (y_valid == 1).sum())

Quantidade classe 0 valid: 63774
Quantidade classe 1 valid: 63774


In [29]:
print('Quantidade classe 0 test:', (y_test == 0).sum())
print('Quantidade classe 1 test:', (y_test == 1).sum())

Quantidade classe 0 test: 63775
Quantidade classe 1 test: 33525


#### Funções auxiliares para métricas

Métricas utilizadas:

- Acurácia

- Recall

- Precision

- F1-Score

- Auroc (Área sob a Curva Roc)

- Matriz de confusão
      [TP  FP]
      [FN  TN]



In [30]:
### VALIDAR: Teste estatístico Kolmogorov-Smirnov -KS (principal)
### TODO: Adiconar alguns plots

In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve, RocCurveDisplay

import matplotlib
import matplotlib.pyplot as plt

In [32]:
def print_metrics(actual, pred, pred_proba):
  print('Accuracy: {:.4f}'.format(accuracy_score(actual, pred)))
  print('Recall: {:.4f}'.format(recall_score(actual, pred)))
  print('Precision: {:.4f}'.format(precision_score(actual, pred)))
  print('F1-Score: {:.4f}'.format(f1_score(actual, pred)))
  print('ROC AUC Score: {:.4f}'.format(roc_auc_score(actual, pred_proba[:, 1])))
  print('Matriz de confusão no conjunto de teste:')
  print(confusion_matrix(actual, pred))

In [33]:
def eval_metrics(actual, pred):
  accuracy = accuracy_score(actual, pred)
  recall = recall_score(actual, pred)
  precision = precision_score(actual, pred)
  f1 = f1_score(actual, pred)
    # adicionar KS
  return accuracy, recall, precision, f1

In [34]:
mlflow.sklearn.autolog()
mlflow.set_experiment('validacao')

<Experiment: artifact_location='file:///home/ubuntu/Desktop/python-machine-learning-kit/redes-neurais/mlruns/1', creation_time=1664471361376, experiment_id='1', last_update_time=1664471361376, lifecycle_stage='active', name='validacao', tags={}>

### Modelo SVM

Descrição dos principais parâmetros:

- C
  - Parâmetro de regularização. A força da regularização é inversamente proporcional a C. Deve ser estritamente positiva. A penalidade é uma penalidade de 12 ao quadrado.

- kernel
  - Especifica o tipo de kernel a ser usado no algoritmo. Se nenhum for fornecido, 'rbf' será usado. Se um callable for fornecido, ele será usado para pré-computar a matriz do kernel a partir de matrizes de dados; essa matriz deve ser uma matriz de forma
  - Valor default: rbf

- degree
  - Grau da função kernel polinomial ('poli'). Ignorado por todos os outros kernels.
  - Valor default: 3

- gamma
  - Coeficiente de kernel para 'rbf', 'poli' e 'sigmóide'


In [93]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto', probability=True)

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_5,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14
0,1,0,1,1,0,0,0,0.512208,1,0.111111,...,1,1,0,0,0,1,0,1,0,1
1,1,1,0,1,0,0,0,0.512833,0,0.111111,...,1,1,1,0,0,1,0,1,1,0
2,1,0,1,0,0,1,0,0.677307,0,0.111111,...,1,0,0,0,0,1,0,1,1,0
3,1,1,0,0,1,0,0,0.064333,1,0.000000,...,0,0,1,1,0,1,0,1,0,0
4,0,1,1,0,0,0,1,0.112892,1,0.111111,...,0,0,1,0,1,1,1,0,1,0


In [ ]:
%%time
# clf_svm.fit(X_train, y_train)

# X_train2 = X_train[0:3]
# y_train2 = [0, 1, 0]
# clf_svm.fit(X_train, y_train)

In [104]:
# y_valid_pred_svm = clf_svm.predict(X_valid)
# y_valid_pred_svm

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# y_valid_proba_svm = clf_svm.predict_proba(X_valid)

##### Análise de Desempenho

In [ ]:
# print('Métricas da 1ª configuração do SVM:\n')
# print_metrics(y_valid, y_valid_pred_svm, y_valid_proba_svm)

**Segundo a própria documentação do Scikit learn para dataset muito grandes é aconselhável usarmos o LinearSVC**

In [35]:
from sklearn.svm import LinearSVC
clf_svm_linear = LinearSVC(random_state=0, tol=1e-5)

In [36]:
%%time
# clf_svm_linear.fit(X_train, y_train)

2022/09/29 18:47:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4ef9cc0f2cab4272b1b1510c507545cf', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/home/ubuntu/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
2022/09/29 18:51:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.7/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) th

CPU times: user 4min 11s, sys: 10.2 s, total: 4min 22s
Wall time: 4min 11s


LinearSVC(random_state=0, tol=1e-05)

In [38]:
# y_valid_pred_svm_linear = clf_svm_linear.predict(X_valid)

##### Análise de Desempenho

In [39]:
print('Métricas da 1ª configuração do SVM SVC Linear:\n')
print_metrics(y_valid, y_valid_pred_svm_linear,None)

Métricas da 1ª configuração do SVM SVC Linear:

Accuracy: 0.7301
Recall: 0.6573
Precision: 0.7692
F1-Score: 0.7089
Matriz de confusão no conjunto de teste:
[[51199 12575]
 [21854 41920]]


### Modelo Random Forest

Descrição dos parâmetros:

- n_estimators
  - O número de árvores na floresta.

- criterion
  - A função para medir a qualidade de uma divisão

- max_depth
  - A profundidade máxima da árvore.
  - 'None' significa que os nós são expandidos até que todas as folhas sejam puras(se o nó possui prediz apenas 1 classe) ou até que todas as folhas contenham menos de min_samples_split amostras.

- min_samples_split
  - O número mínimo de amostras necessárias para dividir um nó.

- min_samples_leaf
  - O número mínimo de amostrar necessárias para ser um nó folha.

- max_features
  - O número de features a serem considerados ao procurar a melhor divisão. Por exemplo, caso a função seja 'sqrt', a cada divisão ele tenta buscar uma condição que possua sqrt(n_node) entradas. 

- max_leaf_nodes
  - O número max de nós folha. 
  - 'None' significa então pode haver um número ilimitado de nós folha.

In [35]:
from sklearn.ensemble import RandomForestClassifier

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- n_estimators = 100

- criterion = 'gini'

- max_depth = None 

- min_samples_split = 2

- min_samples_leaf = 1

- max_features = 'sqrt'

- max_leaf_nodes = None

In [36]:
clf_rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [37]:
%%time
history_rf = clf_rf.fit(X_train, y_train)

2022/09/30 02:16:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '87164fc9014743b9aa8427c7f0b45360', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/09/30 02:17:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.7/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Va

CPU times: user 10min 15s, sys: 6.62 s, total: 10min 22s
Wall time: 42.4 s


In [38]:
y_valid_pred_rf = clf_rf.predict(X_valid)

In [39]:
y_valid_proba_rf = clf_rf.predict_proba(X_valid)

##### Análise de Desempenho

In [40]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_valid_pred_rf, y_valid_proba_rf)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7334
Recall: 0.6455
Precision: 0.7832
F1-Score: 0.7077
ROC AUC Score: 0.8138
Matriz de confusão no conjunto de teste:
[[52380 11394]
 [22607 41167]]


In [47]:
len(X_train)

255098

#### Random Forest usando K-fold Cross Validation

In [67]:
from sklearn.model_selection import KFold

scores = []
# cv = KFold(n_splits=10, random_state=42, shuffle=True)
kfold = KFold(n_splits=3, shuffle=True)

# for train_index, test_index in cv.split(X_train):
#     print("TRAIN:", train_index, "TEST:", test_index, "LEN:", X_train.loc[255096:255097])
#     X_train2, X_test2, y_train2, y_test2 = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
#     clf_rf.fit(X_train2, y_train2)
#     scores.append(clf_rf.score(X_test2, y_test2))

In [68]:
from sklearn.model_selection import cross_val_score

cross_val_score(clf_rf, X_train, y_train, cv=kfold, scoring='accuracy')

array([0.76057531, 0.76244517, 0.76138395])

# 2ª Configuração

Para a segunda configuração vamos criar um estudo de caso usando o optuna, variando alguns hiperparâmetros

In [52]:
def random_forest(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 4, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
    }
    # Create the model
    with mlflow.start_run(run_name="Random Forest - Validacao"):
        rnd_forest = RandomForestClassifier(
            n_estimators=params["n_estimators"],
            max_depth=params["max_depth"],
            min_samples_split=params["min_samples_split"],
            random_state=42,
            n_jobs=-1
        )

        rnd_forest.fit(X_train, y_train)

        y_pred_valid = rnd_forest.predict(X_valid)
        y_pred_proba = rnd_forest.predict_proba(X_valid)

        (accuracy, recall, precision, f1) = eval_metrics(y_valid, y_pred_valid)

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1", f1)

        gc.collect()
        return accuracy

In [53]:
rf = optuna.create_study(direction="maximize")
rf.optimize(random_forest, n_trials=10)

[I 2022-09-29 18:59:27,225] A new study created in memory with name: no-name-60e35998-c952-4007-a110-39236862f3e1
2022/09/29 18:59:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.7/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
[I 2022-09-29 19:00:01,534] Trial 

[I 2022-09-29 19:06:46,148] Trial 7 finished with value: 0.7419246087747358 and parameters: {'n_estimators': 72, 'max_depth': 62, 'min_samples_split': 85}. Best is trial 5 with value: 0.744950920437796.
2022/09/29 19:07:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.7/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handl

### MLP

Descrição dos parâmetros:

- n_estimators
  - O número de árvores na floresta.


In [56]:
%pip install --user tensorflow

Note: you may need to restart the kernel to use updated packages.


In [57]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- n_

In [58]:
def create_sklearn_compatible_model():
    input_dim = X_train.shape[1]
    
    model = Sequential()
    model.add(Dense(20, activation='tanh', input_dim=input_dim))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [59]:
clf_mlp = KerasClassifier(build_fn=create_sklearn_compatible_model, 
                          batch_size=64, epochs=100,
                          verbose=0)


/home/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
%%time
clf_mlp.fit(X_train, y_train)

CPU times: user 18min 39s, sys: 4min 4s, total: 22min 43s
Wall time: 9min 15s


In [61]:
y_pred_mlp = clf_mlp.predict(X_valid)

3986/3986 [==============================] - 4s 969us/step


In [62]:
y_proba_mlp = clf_mlp.predict_proba(X_valid)

3986/3986 [==============================] - 4s 1ms/step


##### Análise de Desempenho

In [63]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_pred_mlp, y_proba_mlp)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7295
Recall: 0.6847
Precision: 0.7521
F1-Score: 0.7168
ROC AUC Score: 0.8149
Matriz de confusão no conjunto de teste:
[[49379 14395]
 [20110 43664]]


#### 2ª Configuração

Para a segunda configuração alteramos os seguintes parâmetros:

### Gradient Boosting

Descrição dos parâmetros:

- loss
  - Possíveis valores: ‘log_loss’, ‘exponential’
  - default=’log_loss’
  - A função de perda a ser otimizada. 
    - 'log_loss' refere-se ao desvio binomial e multinomial, o mesmo usado na regressão logística. É uma boa escolha para classificação com saídas probabilísticas. 
    - 'exponencial', o aumento de gradiente recupera o algoritmo AdaBoost.

- learning_rate
  - Possíveis valores: intervalo (0,0, inf)
  - default=0.1
  - A taxa de aprendizado reduz a contribuição de cada árvore por learning_rate. Há um trade-off entre learning_rate e n_estimators. 

- n_estimators
  - Os valores devem estar no intervalo [1, inf).
  - default=100
  - O número de estágios de reforço a serem executados. 
  - **O aumento de gradiente é bastante robusto ao over-fitting, portanto, um número grande geralmente resulta em melhor desempenho.

- subsample
  - Os valores devem estar no intervalo (0,0, 1,0].
  - default=1.0
  - A fração de amostras a ser usada para ajustar os 'individual base learners'. 
  - Se menor que 1,0, isso resulta em aumento de gradiente estocástico. subamostra interage com o parâmetro n_estimators. 
    - **Escolher subamostra < 1,0 leva a uma redução da variância e a um aumento no viés.

- criterion
  - Possíveis valores: {‘friedman_mse’, ‘squared_error’, ‘mse’}
  - default=’friedman_mse’
  - A função para medir a qualidade de uma divisão. 
  - Os critérios suportados são:
    - ‘friedman_mse’ para o erro quadrático médio com pontuação de melhoria por Friedman, 
    - ‘squared_error’ para erro quadrático médio. 
  - **O valor padrão de ‘friedman_mse’ geralmente é o melhor, pois pode fornecer uma melhor aproximação em alguns casos.

- min_samples_split
  - Possíveis valores: int or float
  - default=2
  - Se int, os valores devem estar no intervalo [1, inf).
  - Se float, os valores devem estar no intervalo (0.0, 1.0] e min_samples_leaf será ceil(min_samples_leaf * n_samples).

- min_samples_leaf
  - Possíveis valores: int or float
  - default=1
  - Isso pode ter o efeito de suavizar o modelo, especialmente na regressão.

- min_weight_fraction_leaf
  - Os valores devem estar no intervalo [0,0, 0,5].
  - default=0.0(As amostras têm peso igual)
  - A fração ponderada mínima da soma total de pesos (de todas as amostras de entrada) necessária para estar em um nó folha. 

- max_depth
  - Os valores devem estar no intervalo [1, inf).
  - default=3
  - Ajuste este parâmetro para melhorar o desempenho; 
    - O melhor valor depende da interação das variáveis de entrada.

- min_impurity_decrease
  - Os valores devem estar no intervalo [0,0, inf).
  - default=0.0
  - Um nó será dividido se esta divisão induzir uma diminuição da impureza maior ou igual a este valor.

- init
  - Possíveis valores: estimator or ‘zero’
  - default=None(é usado um DummyEstimator)
  - Um objeto estimador que é usado para calcular as previsões iniciais. 
  - init tem que fornecer fit e predict_proba. 
  - Se 'zero', as previsões brutas iniciais são definidas como zero. 

- max_features
  - Possíveis valores: {‘auto’, ‘sqrt’, ‘log2’}, int or float
    - Se int, valores devem estar no intervalo  [1, inf).
    - Se float, valores devem estar no intervalo  (0.0, 1.0] and the features considered at each split will be max(1, int(max_features * n_features_in_)).
    - Se f = ‘auto’, ‘sqrt’, ‘log2’, então max_features = f(n_features).
    - Se None, então max_features = n_features.
  - default=None
  - O número de features para considerar quando buscar pelo melhor split.



In [64]:
from sklearn.ensemble import GradientBoostingClassifier

#### 1ª Configuração

A primeira configuração é utilizada com os parâmetros padrões do Sklearn.

Segue abaixo a lista dos principais hiperparâmetros:

- loss
  - default=’log_loss’

- learning_rate
  - default=0.1

- n_estimators
  - default=100

- subsample
  - default=1.0

- criterion
  - default=’friedman_mse’

- min_samples_split
  - default=2

- min_samples_leaf
  - default=1

- min_weight_fraction_leaf
  - default=0.0(As amostras têm peso igual)

- max_depth
  - default=3

- min_impurity_decrease
  - default=0.0

- init
  - default=None(é usado um DummyEstimator)

- max_features
  - default=None(então max_features = n_features.)




In [62]:
clf_gb = xgb.XGBClassifier(random_state=27)

In [63]:
%%time
clf_gb.fit(X_train, y_train)

CPU times: user 15min 46s, sys: 672 ms, total: 15min 47s
Wall time: 40.5 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=27,
              reg_alpha=0, reg_lambda=1, ...)

In [64]:
y_pred_gb = clf_gb.predict(X_valid)

In [65]:
y_proba_gb = clf_gb.predict_proba(X_valid)

##### Análise de Desempenho

In [66]:
print('Métricas da 1ª configuração do Random Forest:\n')
print_metrics(y_valid, y_pred_gb, y_proba_gb)

Métricas da 1ª configuração do Random Forest:

Accuracy: 0.7570
Recall: 0.6686
Precision: 0.8121
F1-Score: 0.7334
ROC AUC Score: 0.8412
Matriz de confusão no conjunto de teste:
[[53909  9865]
 [21135 42639]]


#### 2ª Configuração

Para a segunda configuração alteramos os seguintes parâmetros:

In [60]:
def gradient_boosting(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate ', 0.0001, 0.1, step=0.005),
        'max_depth': trial.suggest_int('max_depth', 4, 100),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300, step=10)
    }
    # 'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
    # Create the model
    with mlflow.start_run(run_name="Gradient Boosting - Validacao"):
        gb_clf_trial = xgb.XGBClassifier(
            learning_rate=params["learning_rate"],
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
            random_state=42
        )
        print("Training model with params", params)
        gb_clf_trial.fit(X_train, y_train)

        y_pred_valid = gb_clf_trial.predict(X_valid)
        y_pred_proba = gb_clf_trial.predict_proba(X_valid)

        (accuracy, recall, precision, f1) = eval_metrics(y_valid, y_pred_valid)

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1", f1)

        gc.collect()
        return accuracy

In [61]:
gb = optuna.create_study(direction="maximize")
gb.optimize(gradient_boosting, n_trials=10)

[I 2022-09-30 03:41:51,757] A new study created in memory with name: no-name-6d9d3dc5-6165-4317-b52f-c71c0067ae07
/home/ubuntu/.local/lib/python3.7/site-packages/optuna/distributions.py:671: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  low=low, old_high=old_high, high=high, step=step


Training model with params {'learning_rate': 0.0751, 'max_depth': 5, 'n_estimators': 190}


[I 2022-09-30 03:42:56,244] Trial 0 finished with value: 0.7599570357826073 and parameters: {'learning_rate ': 0.0751, 'max_depth': 5, 'n_estimators': 190}. Best is trial 0 with value: 0.7599570357826073.
/home/ubuntu/.local/lib/python3.7/site-packages/optuna/distributions.py:671: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  low=low, old_high=old_high, high=high, step=step


Training model with params {'learning_rate': 0.08510000000000001, 'max_depth': 88, 'n_estimators': 110}


[I 2022-09-30 03:49:04,054] Trial 1 finished with value: 0.7464248753410481 and parameters: {'learning_rate ': 0.08510000000000001, 'max_depth': 88, 'n_estimators': 110}. Best is trial 0 with value: 0.7599570357826073.
/home/ubuntu/.local/lib/python3.7/site-packages/optuna/distributions.py:671: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  low=low, old_high=old_high, high=high, step=step


...

[I 2022-09-30 04:50:08,844] Trial 7 finished with value: 0.7448411578386176 and parameters: {'learning_rate ': 0.015099999999999999, 'max_depth': 94, 'n_estimators': 300}. Best is trial 0 with value: 0.7599570357826073.
/home/ubuntu/.local/lib/python3.7/site-packages/optuna/distributions.py:671: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  low=low, old_high=old_high, high=high, step=step


Training model with params {'learning_rate': 0.0751, 'max_depth': 7, 'n_estimators': 270}


[I 2022-09-30 04:52:16,947] Trial 8 finished with value: 0.7615407532850378 and parameters: {'learning_rate ': 0.0751, 'max_depth': 7, 'n_estimators': 270}. Best is trial 8 with value: 0.7615407532850378.
/home/ubuntu/.local/lib/python3.7/site-packages/optuna/distributions.py:671: UserWarning: The distribution is specified by [0.0001, 0.1] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.0001, 0.0951].
  low=low, old_high=old_high, high=high, step=step


Training model with params {'learning_rate': 0.015099999999999999, 'max_depth': 46, 'n_estimators': 200}


[I 2022-09-30 05:05:07,544] Trial 9 finished with value: 0.7403408912723053 and parameters: {'learning_rate ': 0.015099999999999999, 'max_depth': 46, 'n_estimators': 200}. Best is trial 8 with value: 0.7615407532850378.
